In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("/content/domain_specific_chatbot_data.csv")

In [4]:
df

,query,response,intent,domain
0,What are the side effects of the COVID-19 vacc...,Common side effects of the COVID-19 vaccine in...,side effects inquiry,healthcare
1,How can I schedule an appointment with my doctor?,You can schedule an appointment by calling our...,appointment booking,healthcare
2,What should I do if I miss a dose of my medica...,"If you miss a dose, take it as soon as you rem...",medication inquiry,healthcare
3,How can I check my account balance?,You can check your balance by logging into you...,balance inquiry,finance
4,What is the interest rate for a personal loan?,The current interest rate for personal loans i...,loan inquiry,finance
...,...,...,...,...
2995,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance
2996,What are the symptoms of flu?,"Flu symptoms include fever, cough, sore throat...",flu symptoms inquiry,healthcare
2997,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
2998,Can I make changes to my loan repayment schedule?,Changes to your loan repayment schedule can be...,loan repayment adjustment,finance


In [5]:
from transformers import T5Tokenizer,T5ForConditionalGeneration,Trainer,TrainingArguments

In [6]:
df['response'][0]

'Common side effects of the COVID-19 vaccine include soreness at the injection site, fever, and fatigue.'

In [7]:
from sklearn.model_selection import train_test_split
train_df,val_df=train_test_split(df,test_size=0.2,random_state=42)

In [8]:
train_df.shape

(2400, 4)

In [9]:
train_data=train_df.reset_index(drop=True)
validation_data=val_df.reset_index(drop=True)

In [10]:
train_data

,query,response,intent,domain
0,What should I do if I miss a dose of my medica...,"If you miss a dose, take it as soon as you rem...",medication inquiry,healthcare
1,What are the side effects of the COVID-19 vacc...,Common side effects of the COVID-19 vaccine in...,side effects inquiry,healthcare
2,What are the symptoms of flu?,"Flu symptoms include fever, cough, sore throat...",flu symptoms inquiry,healthcare
3,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
4,What are the side effects of the COVID-19 vacc...,Common side effects of the COVID-19 vaccine in...,side effects inquiry,healthcare
...,...,...,...,...
2395,Can I make changes to my loan repayment schedule?,Changes to your loan repayment schedule can be...,loan repayment adjustment,finance
2396,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance
2397,What are the side effects of the COVID-19 vacc...,Common side effects of the COVID-19 vaccine in...,side effects inquiry,healthcare
2398,What is the interest rate for a personal loan?,The current interest rate for personal loans i...,loan inquiry,finance


In [11]:
import re
def clean_text(text):
    text = re.sub(r'\r\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'<.*?>', '', text)
    text = text.strip().lower()
    return text


In [12]:
train_data['query']=train_data['query'].apply(clean_text)
train_data['response']=train_data['response'].apply(clean_text)

In [13]:
validation_data['query']=validation_data['query'].apply(clean_text)
validation_data['response']=validation_data['response'].apply(clean_text)

In [14]:
tokenizer=T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
def preprocess_function(examples):
  inputs=tokenizer(examples['query'],padding="max_length",truncation=True,max_length=250)
  outputs=tokenizer(examples['response'],padding="max_length",truncation=True,max_length=250)
  inputs['labels']=outputs['input_ids']
  return inputs
train_dataset=train_data.apply(preprocess_function,axis=1)
val_dataset=validation_data.apply(preprocess_function,axis=1)

In [16]:
train_dataset[0]

{'input_ids': [125, 225, 3, 23, 103, 3, 99, 3, 23, 3041, 3, 9, 6742, 13, 82, 7757, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model=T5ForConditionalGeneration.from_pretrained('t5-base')
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=50,
    save_steps=500,
    eval_steps=50,
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kattapranavreddy (kattapranavreddy-anurag-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,13.242300
100,7.426300
150,1.439600
200,0.183400
250,0.069700
300,0.028300
350,0.013900
400,0.007200
450,0.004600
500,0.003200


TrainOutput(global_step=1800, training_loss=0.6231648543213184, metrics={'train_runtime': 1849.7626, 'train_samples_per_second': 7.785, 'train_steps_per_second': 0.973, 'total_flos': 4281735168000000.0, 'train_loss': 0.6231648543213184, 'epoch': 6.0})

In [18]:
import transformers
print(transformers.__version__)


4.51.3


In [19]:
model_save_path = "./t5-finetuned"
trainer.save_model(model_save_path)


In [20]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-base')
tokenizer.save_pretrained(model_save_path)


('./t5-finetuned/tokenizer_config.json',
 './t5-finetuned/special_tokens_map.json',
 './t5-finetuned/spiece.model',
 './t5-finetuned/added_tokens.json')

In [21]:
import torch

if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**2
    reserved = torch.cuda.memory_reserved() / 1024**2
    print(f"Memory Allocated: {allocated:.2f} MB")
    print(f"Memory Reserved: {reserved:.2f} MB")
else:
    print("CUDA not available. Running on CPU.")


Memory Allocated: 2567.96 MB
Memory Reserved: 9490.00 MB


In [22]:
import shutil

shutil.make_archive('t5-finetuned', 'zip', model_save_path)

from google.colab import files
files.download('t5-finetuned.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

model_path = "./t5-finetuned"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict(input_text, max_length=100):
    input_text = input_text.strip().lower()
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

input_query = "I've misplaced my credit card—what steps should I take?"
response = predict(input_query)
print("Model response:", response)


Model response: please contact our customer service immediately to report your lost card and request a replacement.


In [24]:
df

,query,response,intent,domain
0,What are the side effects of the COVID-19 vacc...,Common side effects of the COVID-19 vaccine in...,side effects inquiry,healthcare
1,How can I schedule an appointment with my doctor?,You can schedule an appointment by calling our...,appointment booking,healthcare
2,What should I do if I miss a dose of my medica...,"If you miss a dose, take it as soon as you rem...",medication inquiry,healthcare
3,How can I check my account balance?,You can check your balance by logging into you...,balance inquiry,finance
4,What is the interest rate for a personal loan?,The current interest rate for personal loans i...,loan inquiry,finance
...,...,...,...,...
2995,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance
2996,What are the symptoms of flu?,"Flu symptoms include fever, cough, sore throat...",flu symptoms inquiry,healthcare
2997,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
2998,Can I make changes to my loan repayment schedule?,Changes to your loan repayment schedule can be...,loan repayment adjustment,finance


In [30]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def generate_predictions(example_queries, model, tokenizer, device):
    inputs = tokenizer(example_queries, padding=True, return_tensors="pt", truncation=True, max_length=250)

    input_ids = inputs['input_ids'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids, num_beams=5, max_length=250, early_stopping=True)

    predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return predictions

example_queries = validation_data['query'].head(5).tolist()
predictions = generate_predictions(example_queries, model, tokenizer, device)

for query, pred, actual in zip(example_queries, predictions, validation_data['response'].head(5).tolist()):
    print(f"Query: {query}\nPrediction: {pred}\nActual: {actual}\n{'-'*50}")


Query: how can i schedule an appointment with my doctor?
Prediction: you can schedule an appointment by calling our office or using our online portal.
Actual: you can schedule an appointment by calling our office or using our online portal.
--------------------------------------------------
Query: what are the side effects of the covid-19 vaccine?
Prediction: common side effects of the covid-19 vaccine include soreness at the injection site, fever, and fatigue.
Actual: common side effects of the covid-19 vaccine include soreness at the injection site, fever, and fatigue.
--------------------------------------------------
Query: how do i update my contact details on my account?
Prediction: to update your contact details, log into your account and go to the 'profile' section.
Actual: to update your contact details, log into your account and go to the 'profile' section.
--------------------------------------------------
Query: how can i schedule an appointment with my doctor?
Prediction: 

In [32]:
df.tail()

,query,response,intent,domain
2995,"I lost my credit card, what should I do?",Please contact our customer service immediatel...,lost card reporting,finance
2996,What are the symptoms of flu?,"Flu symptoms include fever, cough, sore throat...",flu symptoms inquiry,healthcare
2997,How do I update my contact details on my account?,"To update your contact details, log into your ...",contact update,finance
2998,Can I make changes to my loan repayment schedule?,Changes to your loan repayment schedule can be...,loan repayment adjustment,finance
2999,How do I apply for a student loan?,You can apply for a student loan by visiting o...,student loan application,finance


In [38]:
df[df['query']=='How do I update my contact details on my account?']['response']

,response
7,"To update your contact details, log into your ..."
17,"To update your contact details, log into your ..."
27,"To update your contact details, log into your ..."
37,"To update your contact details, log into your ..."
47,"To update your contact details, log into your ..."
...,...
2957,"To update your contact details, log into your ..."
2967,"To update your contact details, log into your ..."
2977,"To update your contact details, log into your ..."
2987,"To update your contact details, log into your ..."


In [39]:
response = df[df['query'] == 'How do I update my contact details on my account?']['response'].iloc[0]
print(response)


To update your contact details, log into your account and go to the 'Profile' section.


In [40]:
eval_results = trainer.evaluate()
print(f"Evaluation loss: {eval_results['eval_loss']}")


Evaluation loss: 3.948004632547963e-06


In [44]:
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

def compute_bleu_score(model, tokenizer, eval_dataset, max_length=250):
    model.eval()
    predictions = []
    references = []

    for example in tqdm(eval_dataset):
        input_text = example['query']
        actual_output = example['response']

        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        input_ids = inputs['input_ids'].to(model.device)

        output_ids = model.generate(input_ids)

        predicted_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        predictions.append(predicted_text)
        references.append([actual_output.split()])

    bleu_score = 0
    for pred, ref in zip(predictions, references):
        bleu_score += sentence_bleu(ref, pred.split())
    bleu_score /= len(predictions)
    return bleu_score

val_dataset = validation_data.to_dict(orient='records')
bleu_score = compute_bleu_score(model, tokenizer, val_dataset)
print(f"BLEU Score: {bleu_score}")


100%|██████████| 600/600 [03:38<00:00,  2.75it/s]

BLEU Score: 0.9133675743012869
